In [1]:
import sys
sys.path.append('../')

from config import zcorpus_fname, feature_manager_fname
from config import word2vec_model_fname
from config import z_mm_fname, z_mm_xwords_fname, z_mm_feature_vocab_fname

from simple_ner import get_process_memory, write_list
from simple_ner import FeatureManager, ZCorpus
from simple_ner import zcorpus_to_sparsematrix

## Prepare training data from zcorpus

In [2]:
zcorpus = ZCorpus(zcorpus_fname)
feature_manager = FeatureManager()
feature_manager.load(feature_manager_fname)

for i,z in enumerate(zcorpus):
    if i > 5: break
    print(z)

('아뇨', ['4428'])
('내꺼', ['1666', '1637', '30215'])
('아닌데', ['1798', '2783', '185588', '1'])
('요', ['4007', '2202'])
('네', ['8514'])
('은이', ['75', '360453'])


In [40]:
x, int2word, feature_vocab = zcorpus_to_sparsematrix(zcorpus,
                                                     feature_manager,
                                                     ner_min_feature_count=20,
                                                     pruning_per_instance=1000000,
                                                     pruning_min_featuer_count=2)

transforming zcorups to sparse matrix was done. #words= 69780 mem= 6.176 Gb
It returns (x, int2word)


x.shape = (# words, # features)

In [41]:
x.shape

(69780, 443640)

int2word: words corresponding row of x

In [42]:
len(int2word)

69780

save
- sparse matrix: x (type: scipy.sparse.csr_matrix)
- row indexer: int2word (type: list of str)
- column indexer: feature_vocab (type: list of str)

In [43]:
from scipy.io import mmwrite
mmwrite(z_mm_fname, x)
write_list(z_mm_xwords_fname, int2word)
write_list(z_mm_feature_vocab_fname, feature_vocab)

반드시 row normalize 해야 함. row norm = 1이 되도록 L2 norm. 하지 않을 경우 Logistic Regression의 coefficient가 scale effect 받음

In [79]:
from sklearn.preprocessing import normalize
x = normalize(x)

## Training iter = 1

초기 seed words

In [82]:
ner_seeds = '햄벅  빈대떡  팥죽  채소  인스턴트  고로케  홍어  크레페  치폴레  수제비  떡튀순  문어  콜라  삼김  짜왕  꽃게  복숭아  요구르트  쌈도  야식  한우  칼국수  해물찜  쌀밥  육개장  골뱅이  후식  아구찜  컵밥  음료수  고기  낙지  유자차  조개구이  비비큐  꼬치  케익  꼬기  청국장  갈비탕  닭발  피자  킹크랩  견과류  에스프레소  훠궈  샤브  엽떡  쫄면  아점  상추  소스  가래떡  석식  쿨피스  삼치  프링글스  카레  식혜  키위  맥주  석류  짜파게티  순대국  씨리얼  컵라면  초밥  껍데기  와플  비빔밥  치킨  음료  헛개수  뼈해장국  등갈비  갈비  두부  핫도그  라멘  잡채밥  파닭  순대  밀크티  호떡  치맥  미숫가루  떡볶이  소고기  햄버거  갈비찜  닭도  포카리  불고기  딤섬  타코야끼  김밥  탕수육  미역국  먹이  장어  갈매기살  청심환  삼겹살  광어  삼겹  수박  곱창  핫식스  풀떼기  팥빙수  마늘  쌈은  간식  주먹밥  김치찌개  닭강정  식초  월남쌈  국밥  브런치  블루베리  양꼬치  감자  홈런볼  호박죽  우유  쟁반짜장  스무디  빙수  쌈이랑  맥모닝  쌀국수  소주  과일  싸이버거  고등어  감튀  밀가루  소맥  컵누들  햇반  분식  닭갈비  떡갈비  족발  쉐이크  닭도리탕  반마리  바나나  보드카  고구마  분유  환타  쥐포  참외  뻥튀기  홍시  해장국  감자탕  살안찌  육포  소세지  양파  버블티  송편  깐풍기  파전  순두부  브리또  옥수수  츄러스  쥬스  치즈  짜장면  소금구이  쌈이  찜닭  백숙  곰국  콩국수  스시  라묜  팝콘  닭백숙  돈까스  야채  오징어  참치  연어  커리  빅맥  새우  삼계탕  토마토  닭꼬치  닭가슴살  간장게장  칙힌  갈치  물회  냉면  꼼장어  오뎅탕  돈부리  생선구이  티라미수  파스타  청하  타코  군만두  떡국  베지밀  설렁탕  아이스크림  북어국  라볶이  게장  순하리  몽쉘  쭈꾸미  추어탕  해물탕  점심  디저트  체리  샤브샤브  누룽지  꼬막  짬뽕  홍삼  샌드위치  조개찜  박카스  우동  육회  와퍼  핫바  바베큐  부대찌개  과메기  과자  수제버거  쌍화탕  밀면  꽃게탕  사과주스  바닐라라떼  샐러드  오리  비빔면  비요뜨  탕슉  목살'.split('  ')
print(ner_seeds)
print(len(ner_seeds))

['햄벅', '빈대떡', '팥죽', '채소', '인스턴트', '고로케', '홍어', '크레페', '치폴레', '수제비', '떡튀순', '문어', '콜라', '삼김', '짜왕', '꽃게', '복숭아', '요구르트', '쌈도', '야식', '한우', '칼국수', '해물찜', '쌀밥', '육개장', '골뱅이', '후식', '아구찜', '컵밥', '음료수', '고기', '낙지', '유자차', '조개구이', '비비큐', '꼬치', '케익', '꼬기', '청국장', '갈비탕', '닭발', '피자', '킹크랩', '견과류', '에스프레소', '훠궈', '샤브', '엽떡', '쫄면', '아점', '상추', '소스', '가래떡', '석식', '쿨피스', '삼치', '프링글스', '카레', '식혜', '키위', '맥주', '석류', '짜파게티', '순대국', '씨리얼', '컵라면', '초밥', '껍데기', '와플', '비빔밥', '치킨', '음료', '헛개수', '뼈해장국', '등갈비', '갈비', '두부', '핫도그', '라멘', '잡채밥', '파닭', '순대', '밀크티', '호떡', '치맥', '미숫가루', '떡볶이', '소고기', '햄버거', '갈비찜', '닭도', '포카리', '불고기', '딤섬', '타코야끼', '김밥', '탕수육', '미역국', '먹이', '장어', '갈매기살', '청심환', '삼겹살', '광어', '삼겹', '수박', '곱창', '핫식스', '풀떼기', '팥빙수', '마늘', '쌈은', '간식', '주먹밥', '김치찌개', '닭강정', '식초', '월남쌈', '국밥', '브런치', '블루베리', '양꼬치', '감자', '홈런볼', '호박죽', '우유', '쟁반짜장', '스무디', '빙수', '쌈이랑', '맥모닝', '쌀국수', '소주', '과일', '싸이버거', '고등어', '감튀', '밀가루', '소맥', '컵누들', '햇반', '분식', '닭갈비', '떡갈비', '족발', '쉐이크', '닭도리탕', '반마리', '바나나', '보드카', '고구마

word2int를 이용한 ner seed의 id 화

In [46]:
word2int = {w:i for i,w in enumerate(int2word)}
ner_seeds_idx = {word2int[w] for w in ner_seeds if w in word2int}
len(ner_seeds_idx)

253

seed words를 positive label, 나머지를 negative label로 만듦

In [50]:
y = [1 if i in ner_seeds_idx else -1 for i in range(x.shape[0])]

In [75]:
%%time

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(penalty='l2', C=1.0, class_weight={1:0.98, -1:0.02})
logistic.fit(x, y)
prob = logistic.predict_proba(x)

CPU times: user 11.4 s, sys: 0 ns, total: 11.4 s
Wall time: 11.4 s


two class (-1, 1)이기 때문에 positive class가 2번째 클래스. prob에서 [:,1]을 가져오면 positive class probability가 나옴. 이를 decreasing order로 정렬하여 int2word로 look-up

In [76]:
extracted_ners = sorted(enumerate(prob[:,1]), key=lambda x:x[1], reverse=True)
extracted_ners = [(int2word[i], p) for i, p in extracted_ners]

In [85]:
' '.join([word for word, p in extracted_ners[:1000] if not (word in ner_seeds)])

'스파게티 보쌈 냉모밀 닭죽 라면 바나나우유 순두부찌개 짱깨 콘푸로스트 된장찌개 초콜릿 진통제 팟타이 소바 젤리 마카롱 알탕 잡채 밥 떡만두국 귤 맘마 간단히 빵도 사케 맛난거 한정식 닭한마리 해열제 짜파구리 닭계장 갈치조림 매화수 차돌박이 선식 군고구마 뿌링클 곶감 칸쵸 파인애플 곤드레밥 통닭 까르보나라 맛잇는거 전복죽 육회비빔밥 약 두루치기 허니버터칩 학식 밥만 뚝불 에그타르트 씨앗호떡 달달한거 딸기 오뎅 빕 전어 떡꼬치 나쵸 양념갈비 한라봉 떡뽀끼 술처 계란후라이 오꼬노미야끼 뭐좀 즉떡 밥비벼 항정살 옻닭 수면제 문어숙회 보신탕 간단하게 프레즐 약안 매콤한거 보리밥 동태탕 얻어 대게 오돌뼈 메밀소바 집밥 회 밥안 보충제 지코바 알밥 콩불 거봉 약도 양념게장 콘프레이크 아수쿠림 빠네 닭똥집 빵 고등어조림 콘푸라이트 호빵 물만 가츠동 딸바 마라샹궈 스팸 곰장어 징거버거 와구와구 빵좀 이슬만 매운갈비찜 비냉 선지해장국 꿔바로우 만둣국 부침개 와인 도시락 송어회 너구리 빠삐코 귤도 수육 파니니 죠떡 떡 내장탕 말랑카우 생선 스크류바 젤라또 고등어구이 유산균 타르트 밥좀 샤부샤부 라자냐 떡보끼 오믈렛 한약 치돈 꼬깔콘 전어회 큰맘 약좀 만듀 도지마롤 조개 모밀 쭈삼 리조또 전복 밥다 백반 항생제 사시미 양배추 칵테일 자몽주스 회도 술 죽만 선지국 오트밀 사과 빵만 피쟈 뭐쫌 코다리찜 꿀떡 두통약 사탕 굴보쌈 포카칩 아점으로 만두 뭐랑 뼈다귀해장국 철분제 줏어 웨하스 하리보 도넛 식빵 삼치구이 보쌈정식 레모나 부추전 N그릇 동까쓰 맘스터치 국수 밥두안 두그릇 장조림 쌈밥 번데기 곰탕 동동주 봉구스 육사시미 태국음식 난술 사케동 국물있는거 비타민 퀘사디아 물냉 밥못 한입 버거킹 방토 기내식 술안 콩비지 건빵 영양제 생태탕 군밤 한식 수프 두조각 쿠앤크 설빙 뭐 광어회 아몬드 규동 데워서 N조각 양장피 홍합 콘칩 마늘빵 해산물 봉구스밥버거 겔포스 왕창 대하 조식 동그랑땡 야무지게 반계탕 어묵 고량주 급식 밥은 짬밥 등심 크런키 두끼 베라 납작만두 밥이라도 정로환 발포비

## Training iter = 2

extracted_ners에서 단어를 골라 seed words에 추가한 뒤 다시 돌려보자

상위 1000개 중에서 seed words에 포함되지 않은 단어들

In [86]:
'스파게티 보쌈 냉모밀 닭죽 라면 바나나우유 순두부찌개 짱깨 콘푸로스트 된장찌개 초콜릿 진통제 팟타이 소바 젤리 마카롱 알탕 잡채 밥 떡만두국 귤 맘마 간단히 빵도 사케 맛난거 한정식 닭한마리 해열제 짜파구리 닭계장 갈치조림 매화수 차돌박이 선식 군고구마 뿌링클 곶감 칸쵸 파인애플 곤드레밥 통닭 까르보나라 맛잇는거 전복죽 육회비빔밥 약 두루치기 허니버터칩 학식 밥만 뚝불 에그타르트 씨앗호떡 달달한거 딸기 오뎅 빕 전어 떡꼬치 나쵸 양념갈비 한라봉 떡뽀끼 술처 계란후라이 오꼬노미야끼 뭐좀 즉떡 밥비벼 항정살 옻닭 수면제 문어숙회 보신탕 간단하게 프레즐 약안 매콤한거 보리밥 동태탕 얻어 대게 오돌뼈 메밀소바 집밥 회 밥안 보충제 지코바 알밥 콩불 거봉 약도 양념게장 콘프레이크 아수쿠림 빠네 닭똥집 빵 고등어조림 콘푸라이트 호빵 물만 가츠동 딸바 마라샹궈 스팸 곰장어 징거버거 와구와구 빵좀 이슬만 매운갈비찜 비냉 선지해장국 꿔바로우 만둣국 부침개 와인 도시락 송어회 너구리 빠삐코 귤도 수육 파니니 죠떡 떡 내장탕 말랑카우 생선 스크류바 젤라또 고등어구이 유산균 타르트 밥좀 샤부샤부 라자냐 떡보끼 오믈렛 한약 치돈 꼬깔콘 전어회 큰맘 약좀 만듀 도지마롤 조개 모밀 쭈삼 리조또 전복 밥다 백반 항생제 사시미 양배추 칵테일 자몽주스 회도 술 죽만 선지국 오트밀 사과 빵만 피쟈 뭐쫌 코다리찜 꿀떡 두통약 사탕 굴보쌈 포카칩 아점으로 만두 뭐랑 뼈다귀해장국 철분제 줏어 웨하스 하리보 도넛 식빵 삼치구이 보쌈정식 레모나 부추전 N그릇 동까쓰 맘스터치 국수 밥두안 두그릇 장조림 쌈밥 번데기 곰탕 동동주 봉구스 육사시미 태국음식 난술 사케동 국물있는거 비타민 퀘사디아 물냉 밥못 한입 버거킹 방토 기내식 술안 콩비지 건빵 영양제 생태탕 군밤 한식 수프 두조각 쿠앤크 설빙 뭐 광어회 아몬드 규동 데워서 N조각 양장피 홍합 콘칩 마늘빵 해산물 봉구스밥버거 겔포스 왕창 대하 조식 동그랑땡 야무지게 반계탕 어묵 고량주 급식 밥은 짬밥 등심 크런키 두끼 베라 납작만두 밥이라도 정로환 발포비타민 밥으로 겉절이 오뎅국 한알 파르페 칼제비 녹두전 생맥 케밥 소염제 커피 짜파 빈츠 두알 꽃등심 기름진거 저녁 복분자 삶은계란 메론우유 상하이 밥부터 죽도 똥집 매운거 포도 저녀 김떡순 안성탕면 우엉차 비벼 생생우동 깻잎 홍차 술만 밥이나 장어구이 풀만 취킨 솜사탕 카푸치노 맥스봉 도스마스 빵또아 데낄라 칠리새우 매생이국 스무디킹 콘프라이트 더위사냥 김찌 위스키 부드러운거 알리오올리오 버섯 닭다리 벤또 구두 칠면조 국물 빼빼로 푸딩 치킹 한조각 맛나게 밥도 대충 선지 짬짜면 우루사 아이스아메리카노 밥잘 기식 쵸콜렛 주전부리 찐빵 떠볶이 밋있게 한조각만 석갈비 밥뭐 몇조각 규카츠 삼걉살 골고루 대추 스낵랩 쌈싸 술더 데워 샌뒤치 아귀찜 맛릿게 망고 녹차아이스크림 우걱우걱 풀때기 깐쇼새우 점저 강정 양식 봉골레 깍두기 게보린 생강차 흰우유 소보루빵 마싯게 허겁지겁 술그만 병식 탕짜면 머핀 빱빠 콘소메 바게트 맞있게 매실 따신거 저녘은 카츠동 미음 죠리퐁 아슈쿠림 인도커리 달다구리 밥또 밥을 대창 한치 주워 N인분 호박고구마 꿀빵 찌짐 적당히 삼교비 수미칩 훈제연어 세그릇 브로콜리 비타오백 쮸꾸미 삼결살 안주 유린기 호식이 홍합탕 맛닜게 호가든 멜라토닌 맥날 양껏 타코와사비 콘치즈 판모밀 웨지감자 한마리 캐슈넛 닭 빵쪼가리 육전 멸치 적게 청양고추 부대에서 주서 산채비빔밥 닭날개 게국지 황태해장국 도가니탕 약은 고추 떡도 오이 약이라도 스미노프 야금야금 훈제오리 야끼우동 부실하게 우유라도 돌솥비빔밥 뚝배기불고기 니트 전어구이 고구마말랭이 델리만쥬 마요네즈 화채 까까 거하게 삼겾살 피임약 탱크보이 사골국 백설기 갈매기 김피탕 도토리묵 토시살 샹그리아 요거트스무디 수면유도제 귀걸이 뜨뜻한거 오이소박이 양대창 산오징어 팔찌 고거 김볶 한끼 냉수 쫀드기 물티슈 날개 허브티 약만 넥타이 회만 생선까스 약과 짜짜로니 빵이라도 빵한조각 머점 맛잏게 원피스 랏츠버거 겹살이 간장계란밥 홍초 시루떡 새콤달콤 골뱅이무침 오니기리 쥬씨 예거 라떼 석화 오코노미야키 샴페인 밥해 많이 호두마루 노가리 왕꿈틀이 칙촉 틴트 구충제 쌀통닭 바지 정식 나시고랭 동태찌게 데자와 샵샵 낮술 닭도리 챙겨 빠에야 향초 허니버터 깨작깨작 조리퐁 멍게 목걸이 그거 돈코츠라멘 구워 무화과 고양이 사발면 시계 찌낀 일식 배추 푸아그라 몇그릇 죽이라도 도너츠 다꼬야끼 호로요이 베토디 스쿨푸드 회충약 뷔페 안까 밥바 피맥 탕짬면 달다구리한거 모히또 뽀글이 사료 깔루아 부추 동태전 누네띠네 맛잇게 샴푸 쌍쌍바 찌개 어묵탕 체리쥬빌레 세제 바퀴벌레 와퍼주니어 대강 N알 팝송 참이슬 냉우동 베스킨 맛동산 도라지 빵을 쿠사리 회를 복스럽게 또뽀끼 갈치구이 동파육 초계탕 불백 전투식량 교통비 육수 차슈동 명태찜 비누 기네스 반지 중식 모과차 지사제 굳게 포카리스웨트 텀블러 지영이 유진이 현미밥 욕 짜장 맥머핀 지원이 효정이 푸짐하게 고추바사삭 복어 공차 북경오리 구어 희진이 팬돌이 꿀물 가볍게 닭가슴 녹차프라푸치노 현정이 메로구이 흰거 약을 키드오 단거 소금 닭을 세조각 묵사발 배즙 앱솔 니꺼 오리주물럭 매실차 아포가토 혜진이 생식 츄파춥스 펑리수 김태희 은영이 황도 치맛살 소연이 밋나게 미역국이랑 수연이 맛읶게 한개 마니 모니터 수정이 맥도날드 쫀디기 나눠 악세사리 거미 누나 휴지 설농탕 로아커 현경이 스킨쉽 파우치 술도 안빼 산삼 팔보채 두부조림 체크카드 메밀전병 아보카도 까자 카누 개불 민선이 버터 본죽 푸지게 떡봌이 생수 버터와플 정은이 예은이 술퍼 슌대국 승연이 선영이 맛탕 뿌셔뿌셔 페리카나 허벅지 맘터 부라더소다 마가렛트 쫄비빔면 반공기만 세진이 외모 끊임없이 수진이 빵빠레 알로에 칠리 눈칫밥 면만 현진이 엉덩이 테라플루 뺏어 곱배기 파파존스 쏘야 하겐다즈 탕볶밥 예거밤 레고 향수 지은이 진영이 피짜 옥동자 N원짜리 회무침 다정이 황태구이 승현이 젓갈 은정이 스릴러 뮤비 혜민이 피카츄 작년에 주연이 빕스 달걀 초꼬렛 서연이 울아빠 액자 혜정이 뼈다귀 N끼 경은이 낙곱새 율무차 스윙칩 송혜교 한스푼 샤오롱바오 영준이'.count(' ')

752

expanded_ner_seed 확장

In [92]:
expanded_ner_seeds = {'와구와구', '젓갈', '쭈삼', '짜장', '샤오롱바오', '라떼', '피임약', '꼬깔콘', '죠떡', '양념갈비', '뿌셔뿌셔', '보신탕', '점저', '약안', '알리오올리오', '넥타이', '일식', '육수', '커피', '리조또', '찐빵', '배즙', '부침개', '풀때기', '틴트', '야끼우동', '훈제연어', '피짜', '피맥', '석갈비', '맥도날드', '더위사냥', '젤라또', '아포가토', '백반', '안주', '허니버터', '벤또', '간장계란밥', '선지국', '동파육', '초계탕', '개불', '매운갈비찜', '급식', '하겐다즈', '국수', '사시미', '파르페', '오니기리', '목걸이', '케밥', '체리쥬빌레', '사탕', '꿔바로우', '현미밥', '크런키', '고량주', '닭', '인도커리', '전어회', '마요네즈', '도라지', '쵸콜렛', '미음', '규동', '강정', '수육', '우엉차', '푸아그라', '만두', '알탕', '스무디킹', '김떡순', '칠리', '생식', '슌대국', '두알', '빵좀', '계란후라이', '만듀', '찌낀', '곰탕', '지사제', '비누', '매운거', '전복', '어묵탕', '송어회', '영양제', '거봉', '칸쵸', '짬짜면', '프레즐', '봉구스', '고추', '전투식량', '고추바사삭', '또뽀끼', '퀘사디아', '닭죽', '부추', '물만', '참이슬', '삼결살', '우루사', '태국음식', '양념게장', '빈츠', '한약', '맥머핀', '예거밤', '양식', '건빵', '삼걉살', '조식', '갈치구이', '겔포스', '와퍼주니어', '한끼', '동태탕', '달다구리', '파니니', '하리보', '호식이', '산오징어', '초콜릿', '밥비벼', '곰장어', '쿠사리', '스낵랩', '구충제', '매생이국', '깔루아', '바나나우유', '오이소박이', '도지마롤', '칠면조', '냉우동', '방토', '쥬씨', '뽀글이', '번데기', '콩비지', '게보린', '스윙칩', '호두마루', '라면', '간단하게', '두끼', '브로콜리', '동태찌게', '콘소메', '굴보쌈', '버섯', '쌈', '바게트', '고구마말랭이', '콘치즈', '기식', '회무침', '짜파', '저녁', '한조각만', '한라봉', '다꼬야끼', '게국지', '명태찜', '한마리', '쏘야', '빵쪼가리', '녹차프라푸치노', '약', '납작만두', '밥안', '찌짐', '반공기만', '콘푸로스트', '도토리묵', '카푸치노', '닭계장', '화채', '장어구이', '규카츠', '마카롱', '우유', '코다리찜', '산채비빔밥', '포카칩', '두루치기', '국물', '맛잇는거', '사과', '떡봌이', '초꼬렛', '닭날개', '닭똥집', '에그타르트', '사케동', '곤드레밥', '빵도', '광어회', '도넛', '칠리새우', '시루떡', '베라', '짜파구리', '아점', '청양고추', '단거', '호로요이', '빠네', '사료', '향초', '본죽', '오돌뼈', '기네스', '밥잘', '훈제오리', '묵사발', '한스푼', '버터', '풀만', '테라플루', '삼치구이', '메밀소바', '수프', '랏츠버거', '샹그리아', '베토디', '토시살', '북경오리', '쿠앤크', '요거트스무디', '망고', '아몬드', '만둣국', '뜨뜻한거', '짱깨', '홍초', '떡보끼', '반계탕', '지코바', '쫀드기', '전복죽', '소바', '국물있는거', '두부조림', '치킹', '오꼬노미야끼', '맘마', '포카리스웨트', '발포비타민', '죽', '비타민', '도스마스', '밥두안', '떡꼬치', '쮸꾸미', '전어구이', '깨작깨작', '세조각', '생태탕', '메론우유', '고등어구이', '한식', '아보카도', '멍게', '호빵', '집밥', '산삼', '부라더소다', '돈코츠라멘', '양배추', '라자냐', '데자와', '빵만', '아수쿠림', '까르보나라', '한입', '떡뽀끼', '학식', '허니버터칩', '생선까스', '대창', '팟타이', '마가렛트', '돌솥비빔밥', '시계', '탱크보이', '날개', '빕', '깍두기', '호가든', '잡채', '스파게티', '꿀빵', '딸기', '빵한조각', '오이', '씨앗호떡', '머핀', '닭가슴', '콘프라이트', '오트밀', '샌뒤치', '짬밥', '향수', '한정식', '차돌박이', '버거킹', '짜짜로니', '뚝배기불고기', '죽이라도', '조리퐁', '두조각', '군밤', '율무차', '내장탕', '공차', '오코노미야키', '동태전', '새콤달콤', '동동주', '피쟈', '김볶', '한조각', '위스키', '칼제비', '너구리', '스미노프', '메로구이', '칵테일', '비냉', '장조림', '원피스', '빵', '쌀통닭', '생맥', '봉구스밥버거', '뼈다귀해장국', '똥집', '떡만두국', '맘터', '로아커', '유린기', '닭다리', '갈치조림', '황태구이', '오믈렛', '예거', '콘푸라이트', '황태해장국', '홍차', '샤부샤부', '오리주물럭', '조개', '차슈동', '샴페인', '떠볶이', '골뱅이무침', '솜사탕', '수면유도제', '알밥', '빕스', '약도', '녹차아이스크림', '왕꿈틀이', '데낄라', '한치', '누네띠네', '안성탕면', '대하', '항생제', '정로환', '냉모밀', '양장피', '바퀴벌레', '따신거', '홍합', '알로에', '구두', '나시고랭', '석화', '뭐랑', '탕볶밥', '뚝불', '멜라토닌', '겉절이', '선지해장국', '달걀', '모밀', '캐슈넛', '오뎅국', '취킨', '허브티', '나쵸', '욕', '배추', '한알', '타르트', '아귀찜', '밥', '생선', '면만', '곱배기', '콘칩', '델리만쥬', '치돈', '스크류바', '갈매기', '스팸', '쌈밥', '스쿨푸드', '빼빼로', '딸바', '와인', '소염제', '수면제', '동까쓰', '쫀디기', '정식', '탕짜면', '기름진거', '빠에야', '매콤한거', '김찌', '귤', '마늘빵', '왕창', '해열제', '대충', '매실', '밥못', '저녀', '봉골레', '쌍쌍바', '생수', '흰우유', '해산물', '무화과', '달달한거', '아슈쿠림', '베스킨', '웨지감자', '탕짬면', '닭도리', '버터와플', '맛동산', '맥스봉', '황도', '오뎅', '복어', '군고구마', '맘스터치', '옻닭', '설농탕', '고등어조림', '대게', '식빵', '죠리퐁', '페리카나', '빵이라도', '맥날', '파파존스', '찌개', '꿀물', '뿌링클', '콩불', '선지', '젤리', '진통제', '징거버거', '기내식', '사골국', '야무지게', '문어숙회', '전어', '보쌈', '타코와사비', '빠삐코', '도가니탕', '육회비빔밥', '텀블러', '불백', '소금', '회충약', '까까', '두통약', '회만', '사케', '보리밥', '선식', '설빙', '메밀전병', '미역국', '철분제', '수미칩', '앱솔', '매화수', '빱빠', '호박고구마', '푸딩', '치맛살', '말랑카우', '팔찌', '맛탕', '매실차', '생강차', '주전부리', '파인애플', '맛난거', '비타오백', '레고', '꽃등심', '마라샹궈', '쫄비빔면', '항정살', '카츠동', '콘프레이크', '냉수', '판모밀', '깻잎', '카누', '삶은계란', '곶감', '두그릇', '적게', '생생우동', '구어', '모과차', '복분자', '홍합탕', '겹살이', '모히또', '가츠동', '비벼', '눈칫밥', '부드러운거', '츄파춥스', '칙촉', '육사시미', '꿀떡', '깐쇼새우', '펑리수', '육전', '어묵', '양대창', '등심', '노가리', '보충제', '바지', '웨하스', '혜정이', '삼겾살', '포도', '부추전', '보쌈정식', '자몽주스', '멸치', '아이스아메리카노', '약좀', '유산균', '낮술', '물냉', '도시락', '녹두전', '중식', '술', '샵샵', '니트', '빵또아', '회', '달다구리한거', '빵빠레', '난술', '통닭', '백설기', '약과', '순두부찌개', '즉떡', '도너츠', '떡', '소보루빵', '사발면', '뷔페', '닭한마리', '팔보채', '상하이', '대추', '된장찌개', '동그랑땡', '김피탕', '레모나'}
expanded_ner_seeds.update(ner_seeds)
print(len(expanded_ner_seeds))

ner_seeds_idx = {word2int[w] for w in expanded_ner_seeds if w in word2int}
y = [1 if i in ner_seeds_idx else -1 for i in range(x.shape[0])]

844


재학습

In [93]:
%%time

logistic = LogisticRegression(penalty='l2', C=1.0, class_weight={1:0.98, -1:0.02})
logistic.fit(x, y)
prob = logistic.predict_proba(x)

CPU times: user 11 s, sys: 0 ns, total: 11 s
Wall time: 11 s


In [94]:
extracted_ners = sorted(enumerate(prob[:,1]), key=lambda x:x[1], reverse=True)
extracted_ners = [(int2word[i], p) for i, p in extracted_ners]

In [97]:
' '.join([word for word, p in extracted_ners[:1500] if not (word in expanded_ner_seeds)])

'간단히 밥만 뭐좀 술처 얻어 이슬만 밥좀 큰맘 귤도 죽만 밥다 뭐쫌 회도 줏어 아점으로 데워서 술안 밥부터 N그릇 밥이라도 뭐 밥이나 밥은 맛나게 밋있게 밥으로 N조각 맛릿게 밥또 마싯게 맞있게 술그만 데워 맛닜게 밥을 밥도 밥뭐 주워 죽도 쌈싸 몇조각 머점 골고루 술더 술만 약이라도 주서 맛잏게 저녘은 우걱우걱 허겁지겁 부실하게 우유라도 챙겨 약은 양껏 야금야금 밥해 적당히 삼교비 세그릇 맛잇게 병식 떡도 약만 거하게 밋나게 안까 구워 굳게 부대에서 N인분 밥바 몇그릇 많이 고거 맛읶게 약을 푸짐하게 복스럽게 빵을 푸지게 안빼 진통제라도 N알 회를 팬돌이 까자 끊임없이 닭을 나눠 마니 밥꼭 미역국이랑 오이냉국 꼬들밥 주는대로 뺏어 뭐안 닥도리탕 다잘 차카니 대강 키드오 귀걸이 술퍼 뽀또 물티슈 깨찰빵 삼겸살 가볍게 양갱 니뽕내뽕 상하이버거 술도 카라아게 오사쯔 그거 N끼 이인분 곱빼기로 옥동자 떡봈이 모유 두공기 쫄볶이 오메기떡 깔루아밀크 빈속에 크로와상 새싹비빔밥 궈 떢복이 꾸역꾸역 와규 짜게 꿀호떡 오쭈 낙곱새 두통약이라도 스콘 새모이만큼 우육탕면 쪼코우유 무침회 떡뽀기 한알만 오미자 익혀서 까스명수 직화구이 잔뜩 암거나 두캔째 자갈치 플레인요거트 맛초킹 흰거 합주실에서 한솥 스릴러 아사히 애껴 해장국이라도 강냉이 세제 뽂음밥 살치살 팔도비빔면 반개만 잡곡밥 라뽂이 구구콘 마들렌 빨대로 떡순튀 토블론 굴비 고양이 뜯어 뼈다귀 칰 북엇국 네스퀵 흰자만 숏다리 술다 악세사리 쿠크다스 김이랑 레모네이드 파게티 너뭐 불량식품 멋난거 앵간히 더위 떠먹는피자 꽈배기 반지 샴푸 우려 위생천 버섯전골 순살파닭 햄구워 식욕억제제 팝송 나물 곱빼기 밥한끼 첵스 모찌 오이무침 꾸이꾸이 N원짜리 대추차 돌솥 단게 땡길때 앗있게 튀김 후드티 머 재첩국 마갈 키위주스 녁 비피더스 스킨쉽 연포탕 진미채 보리차 김태희 컵스프 들깨수제비 국슈 햄바가 짚신 치키니 전투적으로 한그릇 뻔데기 바게뜨 미트볼 와사비 프라푸치노 모랑 녹차 계란 교통비 알약 효정이 고디바 백세주 휴게소에서 모니

## Training iter = 3

In [98]:
expanded_ner_seeds.update(set('오이냉국 꼬들밥 닥도리탕 깨찰빵 삼겸살 양갱 상하이버거 카라아게 오사쯔 떡봈이 모유 쫄볶이 오메기떡 깔루아밀크 크로와상 새싹비빔밥 떢복이 와규 꿀호떡 오쭈 낙곱새 스콘 우육탕면 쪼코우유 무침회 떡뽀기 오미자 까스명수 직화구이 자갈치 플레인요거트 맛초킹 한솥 아사히 강냉이 뽂음밥 살치살 팔도비빔면 잡곡밥 라뽂이 구구콘 마들렌 떡순튀 토블론 굴비뼈다귀 칰 북엇국 네스퀵 숏다리 쿠크다스 레모네이드 파게티 너뭐 불량식품 더위 떠먹는피자 꽈배기 위생천 버섯전골 순살파닭 식욕억제제 나물 곱빼기 밥한끼 첵스 모찌 오이무침 꾸이꾸이 대추차 돌솥 튀김 후드티 재첩국 마갈 키위주스 비피더스 연포탕 진미채 보리차 컵스프 들깨수제비 국슈 햄바가 치키니 뻔데기 바게뜨 미트볼 와사비 프라푸치노 모랑 녹차 계란 알약고디바 백세주 황남빵 피클 돌체라떼 스낵면 후라이드 오곡밥 비아그라 사후피임약 치콜 조기구이 쇠괴기 반숙 인앤아웃 호미빙 포키 메추리알 카라티 공화춘 냉소바 토시래 피로회복제 슐 명란젓 나밥 뽀로로 크루아상 대창도 가치 근육이완제 단화 크래커 껌 민선이 경재랑 허벅지 한라산 콩밥 마포갈매기 송어 나베 쏘시지 정은이 피카츄 꿀자몽 떡벆이 볶짜면 스위스 짜짱면 버거 타꼬야끼 온더보더 항암제 떡을 유선이 방향제 술좀 우롱차 홍삼 메론 백종원도시락 빠삐꼬 타미플루 우동사리 웰치스 빱 야끼소바 더덕구이 단걸 라뽀끼 종류 크로크무슈 물한컵 살코기만 진탕 지은이 스탠드 개껌 고사리 고드름 메운탕 햇반 노랑통닭 식용유 삼인분 피규어 종합비타민 반주 오골계 표고버섯 여명 오동통면 돈깨스 캡사이신 치느님 짱구 마테차 단무지 밑반찬 캔맥 마가리타 음식 돈카츠 몽슈슈 짜글이 쌈채소 팬티 인절미 칭따오 어묵국 찐만두 궁물 맥너겟 크라제 뚝배기 교촌 문어다리 의령소바 유가네 미나리 후르츠 자두 슈니첼 시금치 꽁치 찐고구마찰떡아이스 삼다수  클라우드 깡소주 삼합 똠양꿍 밑반찬 쌈장 메밀전 곤약 우럭 굴소스 케찹 연근 야쿠르트 뼈있는거 도라에몽 요리 덮밥 뼈찜'))
expanded_ner_seeds.update(ner_seeds)
print(len(expanded_ner_seeds))

ner_seeds_idx = {word2int[w] for w in expanded_ner_seeds if w in word2int}
y = [1 if i in ner_seeds_idx else -1 for i in range(x.shape[0])]

1147


재학습

In [147]:
%%time

logistic = LogisticRegression(penalty='l2', C=1.0, class_weight={1:0.90, -1:0.10})
logistic.fit(x, y)
prob = logistic.predict_proba(x)

extracted_ners = sorted(enumerate(prob[:,1]), key=lambda x:x[1], reverse=True)
extracted_ners = [(int2word[i], p) for i, p in extracted_ners]

CPU times: user 13.7 s, sys: 4 ms, total: 13.7 s
Wall time: 13.7 s


아직도 seed set에 넣으면 좋을 거 같은 단어들이 있긴 있다. 하지만 여기서 training은 끝내보자

In [148]:
' '.join([word for word, p in extracted_ners[:1500] if not (word in expanded_ner_seeds)])

'간단히 밥만 술처 뭐좀 얻어 회도 귤도 이슬만 큰맘 밥좀 밥다 뭐쫌 죽만 밥은 아점으로 줏어 데워서 술안 밥으로 밥이라도 우걱우걱 N그릇 저녘은 밥부터 약은 밥도 많이 죽도 떡도 술만 N조각 밥이나 맛나게 맛릿게 밋있게 술더 주서 몇조각 병식 골고루 까자 술그만 밥을 맞있게 야금야금 주워 궈 데워 삼교비 마싯게 쌀 밥뭐 맛닜게 약만 밥또 N알 허겁지겁 밥바 몇그릇 머점 양껏 밥해 우유라도 맛잏게 부실하게 적당히 구워 머 굳게 약이라도 쌈싸 세그릇 챙겨 거하게 마니 맛읶게 빵을 안빼 회를 고거 오메기떡 숭 니뽕내뽕 팬돌이 안까 술도 너뭐 부대에서 꼬들밥 오이냉국 가볍게 양갱 녁 맛잇게 푸지게 잔뜩 밋나게 나물 푸짐하게 N인분 연포탕 가게에서 미역국이랑 키드오 새모이만큼 약을 꿀자몽 구구콘 떡뽀기 잡곡밥 성 다잘 오사쯔 차카니 뭐안 복스럽게 삼겸살 닭을 끊임없이 쓰 단게 나눠 피클 뺏어 네스퀵 크로와상 휴게소에서 대강 살치살 돌솥 팔도비빔면 밥꼭 빈속에 깨찰빵 한솥 를 빵은 나베 파게티 오미자 진통제라도 합주실에서 익혀서 술퍼 옥동자 물티슈 라뽂이 미피 스콘 세잔 뽀또 N끼 프라푸치노 계란 다까 스낵면 그거 레모네이드 또 같이 밥운 이런거 햄바가 닥도리탕 나물밥 까스명수 뮈 겐로쿠 경재랑 떡봈이 세제 버섯전골 새싹비빔밥 오이무침 얌얌 김이랑 크루아상 쨩 튀김 뽂음밥 모유 나밥 뀰 떡순튀 짜게 쫄볶이 아사히 덮밥 애껴 치키니 떢복이 두캔째 쪼코우유 토시래 무도보면서 노란색 굴비 곱빼기로 와사비 순살파닭 마갈 대추차 바게뜨 와규 더덕구이 세명이서 꾸역꾸역 오쭈 안 뻔데기 카라아게 걍안 상하이버거 가족들이랑 낙곱새 더위 보리차 개껌 멋난거 우육탕면 두공기 먹긴 김에다 꽃 전안 모랑 꽈배기 가치 밥한끼 술다 술을 강냉이 한모금 돌체라떼 비피더스 빨간색 들깨수제비 뺀찌 물좀 해장국이라도 비아그라 꿀호떡 긩 떠먹는피자 진미채 한그릇 노랑통닭 스릴러 찡 우려 급하게 이인분 레드벨벳 자갈치 국슈 주는대로 원없이 남은거 산적 돈카츠 영 반개만 넹 마들렌 윤 피치 앗있게 그릇 빠

## Apply trained model to all data

위에서는 3번에 걸쳐 seed words를 확장하며 classifier를 학습시킴. 이를 모든 데이터에 적용해보자. regression에 넣을 필요 없이 학습된 모델의 parameter를 가져와서 내가 직접 extractor를 만들 수 있다.

### 그런데 잘 안된다? 

일반화해서 NER 이 잘 될 줄 알았는데, 아직은 잘 안되네요. 왜 일반화는 잘 안되는지 원인부터 고민해볼게요. 그전에는 위의 반복하는 작업으로 NER 늘려갈 수 있을 거 같네요

In [149]:
coefficients = list(logistic.coef_[0])

feature가 제대로 잡혔음을 볼 수 있다 역시 뒤에 먹는다는 이야기랑 맛있다는 이야기가 나오면 음식

In [150]:
[(feature_vocab[j], coef) for j, coef in sorted(enumerate(coefficients), key=lambda x:x[1], reverse=True)[:20]]

[(('X[1]', '먹고'), 7.9521631299741697),
 (('X[1]', '먹어'), 3.9243879768238643),
 (('X[1]', '맛있'), 3.769157686604566),
 (('X[1]', '먹'), 3.306641017158757),
 (('X[1]', '?'), 3.0829738259319974),
 (('X[1]', '좋아'), 2.9621233854687663),
 (('X[1,2]', ('먹', '었어')), 2.9587612355874797),
 (('X[1]', '먹었'), 2.9108292975763632),
 (('X[1]', '현이'), 2.8133845437674423),
 (('X[1,2]', ('먹고', '싶다')), 2.8087261545795319),
 (('X[-1]', '랑'), 2.7130227614788667),
 (('X[1]', '았어'), 2.65640375573411),
 (('X[1]', '먹으러'), 2.4361130899633419),
 (('X[1]', '먹을'), 2.4060327212934167),
 (('X[-1]', '난'), 2.3813275268562903),
 (('X[-1]', '보'), 2.3215542576585384),
 (('X[1]', '사'), 2.1232299736452327),
 (('X[1]', '먹는'), 2.048033634594276),
 (('X[-1]', '그러'), 1.991742013594936),
 (('X[1]', '는데'), 1.9087725616875018)]

그런데 아직 일반화는 잘 안되는데, 일반화해서 extract 하는건 조금 더 생각해 보겠어요

In [141]:
class TrainedLogisticRegressionExtractorFromZcorpus:
    def __init__(self, coefficients):
        self._coef = coefficients
        
    def extract(self, zcorpus):        
        Score = namedtuple('Score', 'score frequency')        
        scores = {}
        _norm = {}
        frequency = {}
        for i, (word, features) in enumerate(zcorpus):
            frequency[word] = frequency.get(word, 0) + 1
            _norm[word] = _norm.get(word, 0) + len(features)
            for feature in features:
                scores[word] = scores.get(word, 0) + self._coef[int(feature)]
            if i % 1000 == 0:
                args = (100*(i+1)/len(zcorpus), '%', i+1, len(zcorpus))
                sys.stdout.write('\rextracting ... %.3f %s (%d in %d)' % args)
        print('\rextracting was done')
        scores = {word:Score(score/_norm[word], frequency[word]) for word, score in scores.items()}
        scores = sorted(scores.items(), key=lambda x:x[1].score, reverse=True)
        return scores
    
    def save(self, fname):
        with open(fname, 'wb') as f:
            params = {'coefficient': self._coef}
            pickle.dump(params, f)

    def load(self, fname):
        with open(fname, 'rb') as f:
            self._coef = pickle.load(f)

trained_extractor = TrainedLogisticRegressionExtractorFromZcorpus(coefficients)
ner_scores = trained_extractor.extract(zcorpus)

extracting was done


In [160]:
' '.join(tuple(zip(*list(filter(lambda x:x[1].frequency > 5, ner_scores))[3000:4000]))[0])

'살치살 막돼먹은 조오금 저스틴이 스미노프 쉬시는거죠 밥도 뮈해 더웠지 와퍼 우뚜케 중앙동이야 어니언링 조거팬츠 잡채 간단하게 산적이랑 술잘안 리조또 계시나요 벗을까 나았구 산채비빔 찰떡아이스 저녀 어땨 진흙구이 공원이야 킹크랩 될만한걸 건가요 므아아 밖애서 줄기세포 바나나 대추차 흐느적흐느적 진미채 빵점이야 활명수 생식 무라카미 징거더블다운 억만이 키위 소고기 첵스 미지근한물 곱창 지사제는 박용석스시 비빔밥 투움바 육포 닭발 부르셨습니까 옥슈수 푸짐하게 튠즈로 어째서 무엇인가요 뵐까요 디퍼런트 치폴레 금시초문인데 햄벅 슐 핫식스 꽂힌거야 휴개소에서 황남빵 하셨습니까 싶으신데요 플로리 베가스에서 몇년만이야 펜잘큐 갈군다고 만듀 조치조치 만성피로인가 계세요 호박전이랑 어죽 고구마 스물여섯이에요 짬뽕 산고야 다받 인후통 십니까 쵸콜렛 보신탕 빅마마 약정기간이 한거야 청수동 영광인데 몸안 매실차 국수 냈는데 한조각 파짐 마감이야 앱솔루트 주까 알마겔 오곡밥 시캬서 벨코즈 곶감 름 문어다리 카노 췌키라웃 샥스핀 샵샵 닭가슴 보는거야 심플한게 캬라멜마끼야또 파르페 누룽지 르타오 말랑카우 계신가요 뿜엇어 오뎅탕 쿠앤크 술다 진저에일 할순잇 한라봉 짜파 슈크림 다씻 되가니 우여곡절 될까요 춥파춥스 초식남이야 옮은건가 귀연척 배배로 석장에서 줏어 위시리 점심 바쁭가봉가 무알콜로 징거버거세트 빠나나우유 톰포드 떡순튀 부다페 몇병이나 부딪혔나 겅미 먕 걸립니까 쌀통닭 빅맥 술퍼 살충제 게걸스럽게 야간근무에요 앵 킨더조이 얻어 홀서빙 건성이야 탈곡기 피셔스마켓 구우면서 만우절이니까 지입 스시 돈카츠 우울우울열매 한약 겨자소스 물귀신작전 요렁거 겨 탕볶밥 갈비 작심일일 긁힌거야 파요 음오아예 브런치 누네띠네 온그야 알긴아 솜씬데 뭐얼 뭐해 넌먹 썬스프레이 카라멜마끼아또 밥다먹 글라스로 샤이바나 석류 베지밀 려봥 곰실씨 껄 버섯구이 츄파춥스 이한치한 왜끊 쟝블랑제리 디지털프라자 짜글이 녹두전 니뽕내뽕 호박고구마 살안빼 육사시미 청국장 과매기 기특한것 순대 하리보 뭐뭐무머무머 불법다운

In [138]:
filtered_ner_scores = list(filter(lambda x:x[1].frequency > 100, ner_scores))

In [154]:
' '.join(tuple(zip(*filtered_ner_scores[:1000]))[0])

'큰맘 낸거야 술처 밥만 반인데 역인데 드셨나요 파죽 데스까 거에여 분이세요 렇겠죠 는거아냐 먹은겨 밥다 예정인데 내용이야 가깝나 짜리야 뭐먹을거에요 셧어요 분이에요 까진데 끝난다고 가신대 겠는걸 전이지 노는거야 먹엇어요 있냐고 간거에요 잇는거야 있는거야 났나요 가라구 뭐먹을거야 묵을까 밥못 당직이야 사주게 온거에요 머그까 명이야 샌거야 했나요 가시는거에요 가는거야 잇으세요 계신거에요 뭐머겅 계시죠 중이야 있나요 만날까요 드시나요 간거예요 없으세요 가두대 안오지 하는거야 있습니까 밥안 오는겨 뭐할꺼에요 했습니까 중인가요 하시는거죠 안오나 말하는거야 드실래요 오신대 할꺼에요 머머겅 타는거야 뭐해요 가도대 보는거지 머거써 볼래요 마실까 않나요 가능하세요 오는거지 할까요 공강이야 할만해요 오라구 산거에요 몇일이야 갈래요 보낼까 먼말이야 맛잇게먹엇어 가지말까 모먹게 잘거에용 확정이야 쓰는거야 빠른데 바뿌지 올래요 몇명이야 산거야 줄까요 몇호야 사주까 뭐에여 추웠지 아닌가 뭐보게 개안나 옵니까 N이야 시킬까 진심이에요 오디양 뭐에용 올꾸야 꾼거야 볼거에요 집이야 밥머거써 바빠요 오디오디 이뿌지 어느쪽이세요 어느거 어딘데요 오디에용 놀구이써 리얼리 오디얌 잘라궁 놓을까요 오디서 대전이야 몇번이야 어떠니 삐진거야 만드는거야 어떤식으로 내리지 아마두 잘꺼야 잘꺼양 잠오지 잘지내시죠 잡을까 맨입으로 안고팡 그르까 잘꾸야 안고프나 자는고야 공짜야 잘려궁 학생이세요 쯤이야 잘거에여 볼꺼에요 심한데 모라셔 않으세요 몇살인데요 몇살이세요 시내야 수원이야 씻엇어요 하시는거에요 뤠 사무실이에요 밥잘 몇살이에요 밖이야 어떠셔 피시방이야 잡니까 맞으시죠 드신거에요 올텨 딜 못보는거야 쌈도 쉬엇어 드시는거에요 되나요 헐이야 끝난거에요 되는거야 밥부터 쯤인데 약안 있으세요 지내지 끝난거야 어떠세요 뵐까요 될까요 싶으세요 몇이에요 쵸 뭐강 드실거에요 맛나게 프시 들을만해 간거야 맛잇게 화내는거야 깨운거야 밥은 편찮으셔 밥좀 아점 볼텨 정마알 모할꼬야 죽만 몇살이야 몇교시야 뢔 먹는거야 맞죠 뭐좀

In [155]:
' '.join(tuple(zip(*filtered_ner_scores[3000:4000]))[0])

'지말 후루룩 느린데 샤 큥 코만 닭먹 특식 분들이 살찌는거 린당 꿈꿀 놀지 노란색 말없 튕 치즈 빨봉 랴 하루 멘 누운 츄 없지 거안 픈건 쓰니 벌어야 보끔밥 럄 부셔 생수 쟈 아지 맛잇 왜욬 좋앗어 레드벨벳 여덜시반 감아야 뎡 블라우스 니드 출장가 해본 오꾸닭 툐 믹스 밀 본데 정동 유노 졌구 작작 꿈 소면 랑만 놀수있 큰애 서 쇠고기 떳나 갓더 오잇 기필코 긴했 고든 리자 진지한데 비너스 게써 안취 니트 탈이야 직해 죱 팔걸 면 쓰 밖인가 팠는데 찮은데 낙 다녔 시금치 안춥 양파 어느나라 인낫어 한두 찌워야 쩔 져버 유가네 썼어 안돌 서있 디는 는 뒈 끗 같단 윤영이 한병반 비빔 잔뜩 말똥말똥해 삼양 봐버 뜨나 뷴 쿠크다스 꿔야 엽 찜해 주량은 로가 계열 왜에 주사맞 꽃을 단단히 줬지 강냉이 렌 그런가 바빴나보네 도차쿠 났을때 해요 사당이야 까서 덤 너볼 서촌 려야지 믄대 소스 멀지 줬 힐게 우럭 지쳤 내로 로코 팸이 짚신 르 인난거 깍 두드러기 겤 어도 맞거든 장판 폰은 자몽 몫까지 말씀드 배구팡 별장 곰인형 가주 냐고 빤 며씨 랏지 보였 삘인데 렸엉 햄 와 는걸 꿍 썼 지겠 묜 뭐가 많앗어 샌드 모모 가실 아팠 땡초 비누 쏨 텅 붙 싶은데 빨간색 작이야 안살 짧지 호피 효은이 늘려야 현경이 은가 닭다리 치 물약 잼나 피치 페라 갔대 쌀 트루 렸는데 산보 냈거든 찔려 콘이 좀하 홀짝홀짝 겹지 보게 파란색 싶단 겝 조림 침 했지 하것지 향수 잘노 츠 짱 간은 대강 앉을자리 책사러 꼼 몇초 스냅백 낀데 프나 부추 법 키커 외모 셔츠 로제 사시 멘티 야겠 사겨라 봤 경은이 나약 옝 팀전 인데 사 걸려 않거든 두우 악세사리 싫거든 러갈 숟가락 병준이 한점 덕 알배 왜이러지 쟝 묘 뚜기 백팩 꼈나 나술 연상 촉 욘 치느님 촌 후식으로 개솔 갓고 님들이 퇵은 까해 곱 약사 콩은 뱀 냐에 냅니다 낮져밤이 라카 며 개인적으로 좋더 탔어 쿠키 옆사 소 수다떨고 뭔말 오이 젤루 그사 쿠우쿠우 옮김 차례지 쯤요 벙 당면 스러워 육수 만에 렸네 약사다 틴트 재은이 철판 사료